# Day19_0: 생성형 AI와 대규모 언어 모델(LLM) 이해 - 정답

## 실습 퀴즈 정답

**난이도 분포**: 기본 3개 / 응용 3개 / 복합 2개 / 종합 2개

---

In [ ]:
# 환경 설정 (모든 퀴즈에서 사용)
import os
import json
import pandas as pd
from dotenv import load_dotenv
from openai import OpenAI

# .env 파일에서 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 설정
client = OpenAI()

print("OpenAI 클라이언트가 설정되었습니다.")

---

## Q1. LLM 개념 이해 (기본) ⭐

**문제**: 다음 빈칸을 채우세요.

1. LLM은 (______) 아키텍처를 기반으로 합니다.
2. GPT는 (______)-Only 구조이고, BERT는 (______)-Only 구조입니다.
3. GPT는 다음 (______)을 예측하는 방식으로 텍스트를 생성합니다.

In [ ]:
# ====================================
# 정답 코드
# ====================================

# 1번 정답: Transformer
# - 2017년 "Attention Is All You Need" 논문에서 등장
# - RNN/LSTM의 순차 처리 한계를 극복
# - 병렬 처리와 Self-Attention으로 장거리 의존성 해결
answer1 = "Transformer"

# 2번 정답: Decoder, Encoder
# - GPT: Decoder-Only - 단방향(왼쪽→오른쪽) 문맥, 텍스트 생성에 특화
# - BERT: Encoder-Only - 양방향 문맥, 문장 이해/분류에 특화
answer2_gpt = "Decoder"
answer2_bert = "Encoder"

# 3번 정답: 토큰(Token)
# - GPT는 Auto-regressive 방식
# - 이전 토큰들을 보고 다음 토큰을 예측
# - 토큰 = 단어, 서브워드, 또는 문자 단위
answer3 = "토큰(Token)"

print(f"1. {answer1}")
print(f"2. GPT: {answer2_gpt}, BERT: {answer2_bert}")
print(f"3. {answer3}")

In [ ]:
# ====================================
# 테스트/검증
# ====================================

assert answer1.lower() == "transformer", "1번 정답이 틀렸습니다."
assert answer2_gpt.lower() == "decoder", "2번 GPT 정답이 틀렸습니다."
assert answer2_bert.lower() == "encoder", "2번 BERT 정답이 틀렸습니다."
assert "토큰" in answer3 or "token" in answer3.lower(), "3번 정답이 틀렸습니다."

print("모든 정답이 맞습니다!")

### 풀이 설명

**접근 방법**:
- 본문 1.2절 "주요 LLM 아키텍처 비교" 표를 참고합니다.
- 각 모델의 구조적 특징을 이해합니다.

**핵심 개념**:

| 모델 계열 | Transformer 구조 | 특징 | 용도 |
|----------|-----------------|------|------|
| GPT | Decoder-Only | 단방향(왼쪽→오른쪽) | 텍스트 생성 |
| BERT | Encoder-Only | 양방향 | 문장 이해/분류 |
| T5 | Encoder-Decoder | Seq2Seq | 번역, 요약 |

**실무 팁**:
- 텍스트 **생성** 작업 → GPT 계열 선택
- 텍스트 **분류/이해** 작업 → BERT 계열 선택
- 텍스트 **변환** 작업 → T5 계열 선택

---

## Q2. 기본 API 호출 (기본) ⭐

**문제**: OpenAI API를 사용하여 "대한민국의 수도는 어디인가요?"라는 질문에 답변을 받으세요.

In [ ]:
# ====================================
# 정답 코드
# ====================================

# OpenAI API 기본 호출
response = client.chat.completions.create(
    model="gpt-4o-mini",  # 사용할 모델 지정
    messages=[
        # role: "user" = 사용자의 질문
        {"role": "user", "content": "대한민국의 수도는 어디인가요?"}
    ]
)

# 응답에서 텍스트 추출
# - response.choices[0]: 첫 번째 응답 선택
# - .message.content: 실제 텍스트 내용
answer = response.choices[0].message.content

print("질문: 대한민국의 수도는 어디인가요?")
print(f"답변: {answer}")

In [ ]:
# ====================================
# 테스트/검증
# ====================================

# 응답에 "서울"이 포함되어야 함
assert "서울" in answer, f"응답에 '서울'이 포함되지 않았습니다: {answer}"

print("정답이 맞습니다! 대한민국의 수도는 서울입니다.")

### 풀이 설명

**접근 방법**:
1. `client.chat.completions.create()` 메서드 호출
2. `model` 파라미터로 사용할 모델 지정
3. `messages` 파라미터로 대화 내용 전달

**핵심 개념**:

```python
response = client.chat.completions.create(
    model="gpt-4o-mini",     # 모델 선택
    messages=[               # 메시지 리스트
        {"role": "user", "content": "질문"}
    ]
)
```

**응답 구조**:
- `response.choices[0]`: 첫 번째 응답 (보통 하나만 생성)
- `.message.content`: 실제 텍스트 응답

**실무 팁**:
- 간단한 질문에는 `gpt-4o-mini`가 비용 효율적
- 복잡한 추론이 필요하면 `gpt-4o` 사용

---

## Q3. 파라미터 이해 (기본) ⭐⭐

**문제**: temperature를 0.0과 1.5로 각각 설정하여 같은 질문("창의적인 회사 이름 하나 제안해주세요")에 대한 응답을 비교하세요.

In [ ]:
# ====================================
# 정답 코드
# ====================================

prompt = "창의적인 회사 이름 하나 제안해주세요."

# Temperature 0.0: 결정적(deterministic) 응답
# - 항상 가장 확률이 높은 토큰 선택
# - 일관되고 예측 가능한 결과
response_low = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.0,  # 낮은 온도: 결정적
    max_tokens=100
)

# Temperature 1.5: 창의적(creative) 응답
# - 확률 분포가 더 평평해져 다양한 토큰 선택 가능
# - 더 다양하고 예상치 못한 결과
response_high = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=1.5,  # 높은 온도: 창의적
    max_tokens=100
)

print("=== Temperature 비교 ===")
print(f"\n[Temperature 0.0 - 결정적]")
print(response_low.choices[0].message.content)
print(f"\n[Temperature 1.5 - 창의적]")
print(response_high.choices[0].message.content)

In [ ]:
# ====================================
# 테스트/검증 (여러 번 실행하여 차이 확인)
# ====================================

# Temperature 0.0은 여러 번 실행해도 같은 결과
results_low = []
for i in range(3):
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=100
    )
    results_low.append(resp.choices[0].message.content)

print("=== Temperature 0.0 - 3회 실행 ===")
for i, result in enumerate(results_low, 1):
    print(f"{i}. {result[:50]}..." if len(result) > 50 else f"{i}. {result}")

# 모두 같은 결과인지 확인
if len(set(results_low)) == 1:
    print("\nTemperature 0.0: 모든 결과가 동일합니다 (결정적)")
else:
    print("\nTemperature 0.0: 결과가 다릅니다 (API 특성에 따라 약간의 변동 가능)")

### 풀이 설명

**접근 방법**:
1. 동일한 질문으로 temperature만 다르게 설정
2. 각각의 응답을 비교

**핵심 개념**:

| Temperature | 특성 | 권장 용도 |
|-------------|------|----------|
| 0.0 | 결정적, 일관된 결과 | 코드 생성, 데이터 분석 |
| 0.5~0.7 | 균형 잡힌 창의성 | 일반 대화 |
| 1.0~1.5 | 높은 창의성, 다양성 | 브레인스토밍, 창작 |
| 1.5~2.0 | 매우 창의적 (불안정할 수 있음) | 실험적 용도 |

**실무 팁**:
- 분류/추출 작업 → `temperature=0.0`
- 창의적 작업 → `temperature=0.8~1.2`
- 너무 높으면 (>1.5) 결과가 불안정해질 수 있음

---

## Q4. 응답 파싱 (응용) ⭐⭐

**문제**: API 응답에서 다음 정보를 추출하는 코드를 작성하세요.
- 응답 텍스트
- 사용된 총 토큰 수
- 종료 사유(finish_reason)

In [ ]:
# API 호출
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Python의 장점 3가지를 알려주세요."}]
)

In [ ]:
# ====================================
# 정답 코드
# ====================================

# 1. 응답 텍스트 추출
# - choices: 생성된 응답 리스트 (보통 1개)
# - choices[0].message.content: 실제 텍스트
content = response.choices[0].message.content

# 2. 사용된 총 토큰 수 추출
# - usage.prompt_tokens: 입력 토큰 수
# - usage.completion_tokens: 출력 토큰 수
# - usage.total_tokens: 총 토큰 수
total_tokens = response.usage.total_tokens
prompt_tokens = response.usage.prompt_tokens
completion_tokens = response.usage.completion_tokens

# 3. 종료 사유 추출
# - "stop": 자연스럽게 완료
# - "length": max_tokens 도달
# - "content_filter": 콘텐츠 필터링
finish_reason = response.choices[0].finish_reason

print("=== API 응답 파싱 결과 ===")
print(f"\n[응답 텍스트]")
print(content)
print(f"\n[토큰 사용량]")
print(f"  - 입력 토큰: {prompt_tokens}")
print(f"  - 출력 토큰: {completion_tokens}")
print(f"  - 총 토큰: {total_tokens}")
print(f"\n[종료 사유]: {finish_reason}")

In [ ]:
# ====================================
# 테스트/검증
# ====================================

# 응답 텍스트가 비어있지 않은지 확인
assert len(content) > 0, "응답 텍스트가 비어있습니다."

# 토큰 수가 양수인지 확인
assert total_tokens > 0, "토큰 수가 0입니다."
assert prompt_tokens + completion_tokens == total_tokens, "토큰 수 계산이 맞지 않습니다."

# 종료 사유가 유효한지 확인
valid_reasons = ["stop", "length", "content_filter", "tool_calls"]
assert finish_reason in valid_reasons, f"유효하지 않은 종료 사유: {finish_reason}"

print("모든 정보가 올바르게 추출되었습니다!")

### 풀이 설명

**접근 방법**:
1. `response` 객체의 구조 파악
2. 필요한 정보가 있는 위치 접근

**핵심 개념 - 응답 객체 구조**:

```python
response
├── choices: List[Choice]
│   └── [0]
│       ├── message
│       │   ├── role: "assistant"
│       │   └── content: "응답 텍스트"
│       └── finish_reason: "stop"
└── usage
    ├── prompt_tokens: 입력 토큰 수
    ├── completion_tokens: 출력 토큰 수
    └── total_tokens: 총 토큰 수
```

**finish_reason 종류**:

| 값 | 의미 |
|----|------|
| `stop` | 자연스럽게 완료됨 |
| `length` | max_tokens에 도달 |
| `content_filter` | 콘텐츠 정책 위반 |
| `tool_calls` | 함수 호출 필요 |

**실무 팁**:
- `finish_reason="length"`이면 응답이 잘렸을 수 있음 → max_tokens 증가 검토
- 토큰 사용량으로 비용 계산 가능

---

## Q5. 시스템 프롬프트 (응용) ⭐⭐

**문제**: AI가 "친절한 한국어 선생님" 역할을 하도록 시스템 프롬프트를 작성하고, "안녕하세요"의 존댓말 사용법을 질문하세요.

In [ ]:
# ====================================
# 정답 코드
# ====================================

# 시스템 프롬프트: AI의 역할과 행동 방식 정의
system_prompt = """당신은 친절하고 인내심 있는 한국어 선생님입니다.

다음 원칙을 따르세요:
1. 외국인 학습자도 이해하기 쉽게 설명합니다.
2. 예시를 풍부하게 제공합니다.
3. 격려와 칭찬을 아끼지 않습니다.
4. 한국 문화 맥락도 함께 설명합니다."""

# 사용자 질문
user_question = "'안녕하세요'의 존댓말 사용법을 알려주세요. 어떤 상황에서 사용하나요?"

# API 호출
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        # role: "system" - AI의 역할/성격 정의
        {"role": "system", "content": system_prompt},
        # role: "user" - 사용자의 질문
        {"role": "user", "content": user_question}
    ],
    temperature=0.7  # 자연스러운 대화 톤
)

print("=== 한국어 선생님 AI ===")
print(f"\n[시스템 프롬프트]")
print(system_prompt)
print(f"\n[질문]")
print(user_question)
print(f"\n[답변]")
print(response.choices[0].message.content)

In [ ]:
# ====================================
# 테스트/검증
# ====================================

answer = response.choices[0].message.content

# 응답이 충분히 상세한지 확인
assert len(answer) > 100, "응답이 너무 짧습니다. 더 상세한 설명이 필요합니다."

# "안녕하세요" 관련 내용이 포함되어 있는지 확인
assert "안녕" in answer or "인사" in answer, "'안녕' 또는 '인사' 관련 내용이 없습니다."

print("시스템 프롬프트가 잘 적용되었습니다!")

### 풀이 설명

**접근 방법**:
1. 시스템 메시지로 AI의 역할 정의
2. 역할에 맞는 원칙/규칙 명시
3. 사용자 메시지로 질문 전달

**핵심 개념 - 메시지 역할**:

| 역할 | 목적 | 예시 |
|------|------|------|
| `system` | AI 행동 방식 정의 | 역할, 성격, 제약 조건 |
| `user` | 사용자 입력 | 질문, 요청 |
| `assistant` | AI 이전 응답 | 대화 히스토리 |

**좋은 시스템 프롬프트의 요소**:
1. 명확한 역할 정의 ("당신은 ~입니다")
2. 구체적인 행동 원칙
3. 출력 스타일 지정 (필요시)

**실무 팁**:
- 시스템 프롬프트는 대화 전체에 영향
- 너무 길면 토큰 비용 증가
- 핵심 지시사항만 간결하게 작성

---

## Q6. 프롬프트 엔지니어링 (응용) ⭐⭐⭐

**문제**: 다음 텍스트를 "불릿 포인트 3개"로 요약하도록 프롬프트를 작성하세요.

텍스트: "파이썬은 1991년 귀도 반 로섬이 개발한 프로그래밍 언어입니다. 읽기 쉬운 문법과 다양한 라이브러리로 인해 데이터 분석, 웹 개발, 인공지능 등 다양한 분야에서 널리 사용됩니다. 특히 초보자가 배우기 쉬워 프로그래밍 입문 언어로 많이 추천됩니다."

In [ ]:
# ====================================
# 정답 코드
# ====================================

text = """파이썬은 1991년 귀도 반 로섬이 개발한 프로그래밍 언어입니다. 
읽기 쉬운 문법과 다양한 라이브러리로 인해 데이터 분석, 웹 개발, 인공지능 등 
다양한 분야에서 널리 사용됩니다. 특히 초보자가 배우기 쉬워 프로그래밍 입문 
언어로 많이 추천됩니다."""

# 프롬프트 엔지니어링 적용
# 1. 명확한 지시 ("불릿 포인트 3개")
# 2. 출력 형식 지정 ("- " 기호 사용)
# 3. 제약 조건 ("각 항목은 한 문장")
prompt = f"""다음 텍스트를 불릿 포인트 3개로 요약하세요.

규칙:
- 각 항목은 "-" 또는 "*"로 시작
- 각 항목은 한 문장으로 간결하게
- 핵심 정보만 포함

텍스트:
{text}

요약:"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": prompt}],
    temperature=0.3  # 일관된 결과를 위해 낮은 온도
)

print("=== 불릿 포인트 요약 ===")
print(response.choices[0].message.content)

In [ ]:
# ====================================
# 테스트/검증
# ====================================

summary = response.choices[0].message.content

# 불릿 포인트가 포함되어 있는지 확인
bullet_count = summary.count("-") + summary.count("*")
assert bullet_count >= 3, f"불릿 포인트가 3개 미만입니다. 현재: {bullet_count}개"

# 핵심 키워드 포함 확인
keywords = ["파이썬", "Python", "1991", "귀도", "문법", "라이브러리", "초보자", "입문"]
found_keywords = [kw for kw in keywords if kw in summary]
assert len(found_keywords) >= 2, f"핵심 키워드가 부족합니다. 발견: {found_keywords}"

print("요약이 올바르게 생성되었습니다!")
print(f"발견된 키워드: {found_keywords}")

### 풀이 설명

**접근 방법**:
1. 명확한 지시 ("불릿 포인트 3개")
2. 출력 형식 지정 ("-" 기호)
3. 제약 조건 명시 ("한 문장으로")

**핵심 개념 - 프롬프트 엔지니어링 원칙**:

| 원칙 | 적용 예시 |
|------|----------|
| 명확성 | "불릿 포인트 3개로 요약" (숫자 명시) |
| 구조화 | "-"로 시작하도록 형식 지정 |
| 제약 조건 | "각 항목은 한 문장" |
| 맥락 제공 | 원본 텍스트 포함 |

**대안 접근**:
```python
# JSON 형식으로 요청할 수도 있음
prompt = f"""다음 텍스트를 요약하고 JSON으로 반환하세요.
{{"summary": ["포인트1", "포인트2", "포인트3"]}}
텍스트: {text}"""
```

**실무 팁**:
- 요약 작업에는 낮은 temperature (0.0~0.3) 사용
- 출력 형식을 명확히 지정하면 후처리가 쉬움

---

## Q7. Few-shot 프롬프팅 (복합) ⭐⭐⭐

**문제**: Few-shot 방식으로 이메일 분류기를 만드세요. 카테고리: 업무, 광고, 개인

예시를 3개 제공하고, 새로운 이메일 "내일 회의 시간 변경 건으로 연락드립니다."를 분류하세요.

In [ ]:
# ====================================
# 정답 코드
# ====================================

# Few-shot 프롬프트 구성
# 1. 작업 설명
# 2. 예시 3개 (각 카테고리별 1개씩)
# 3. 분류할 새로운 입력
few_shot_prompt = """다음 이메일을 '업무', '광고', '개인' 중 하나로 분류하세요.

예시 1:
이메일: "이번 주 특가! 최대 50% 할인 이벤트를 놓치지 마세요!"
분류: 광고

예시 2:
이메일: "주말에 같이 영화 보러 갈래? 요즘 볼 만한 영화 많더라."
분류: 개인

예시 3:
이메일: "3분기 실적 보고서 검토 부탁드립니다. 금요일까지 피드백 주세요."
분류: 업무

이제 다음 이메일을 분류하세요:
이메일: "내일 회의 시간 변경 건으로 연락드립니다."
분류:"""

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": few_shot_prompt}],
    temperature=0.0  # 일관된 분류를 위해
)

print("=== Few-shot 이메일 분류 ===")
print(f"\n이메일: '내일 회의 시간 변경 건으로 연락드립니다.'")
print(f"분류 결과: {response.choices[0].message.content}")

In [ ]:
# ====================================
# 테스트/검증 (여러 이메일 테스트)
# ====================================

def classify_email(email_text):
    """Few-shot 방식으로 이메일 분류"""
    prompt = f"""다음 이메일을 '업무', '광고', '개인' 중 하나로 분류하세요.

예시 1:
이메일: "이번 주 특가! 최대 50% 할인 이벤트를 놓치지 마세요!"
분류: 광고

예시 2:
이메일: "주말에 같이 영화 보러 갈래? 요즘 볼 만한 영화 많더라."
분류: 개인

예시 3:
이메일: "3분기 실적 보고서 검토 부탁드립니다. 금요일까지 피드백 주세요."
분류: 업무

이제 다음 이메일을 분류하세요:
이메일: "{email_text}"
분류:"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return response.choices[0].message.content.strip()

# 테스트 케이스
test_emails = [
    ("내일 회의 시간 변경 건으로 연락드립니다.", "업무"),
    ("무료 배송! 오늘만 쿠폰 증정!", "광고"),
    ("생일 축하해! 선물 뭐 갖고 싶어?", "개인")
]

print("=== 테스트 결과 ===")
for email, expected in test_emails:
    result = classify_email(email)
    status = "Pass" if expected in result else "Fail"
    print(f"[{status}] '{email[:30]}...' -> {result} (예상: {expected})")

### 풀이 설명

**접근 방법**:
1. 각 카테고리별 대표 예시 선정
2. "이메일: ... 분류: ..." 일관된 형식 사용
3. 마지막에 분류할 이메일 제시

**핵심 개념 - Few-shot 프롬프팅**:

| 방식 | 예시 수 | 특징 |
|------|--------|------|
| Zero-shot | 0개 | 직접 지시만, 간단한 작업에 적합 |
| One-shot | 1개 | 출력 형식 안내에 유용 |
| Few-shot | 2~5개 | 복잡한 분류, 일관된 형식 필요 시 |

**좋은 예시 선정 원칙**:
1. 각 카테고리를 명확히 대표
2. 경계 케이스보다 전형적인 예시
3. 길이와 스타일의 다양성

**실무 팁**:
- 예시가 많을수록 정확도 향상 (비용도 증가)
- 예시 순서는 크게 영향 없지만, 마지막 예시와 비슷한 형식 권장
- 분류 작업에는 `temperature=0.0` 필수

---

## Q8. JSON 출력 (복합) ⭐⭐⭐⭐

**문제**: 다음 제품 리뷰를 분석하여 JSON 형식으로 출력하는 함수를 작성하세요.

출력 형식:
```json
{
    "product": "제품명 추출",
    "rating": 1-5,
    "pros": ["장점1", "장점2"],
    "cons": ["단점1", "단점2"]
}
```

테스트 리뷰: "갤럭시 S24 사용 중인데, 카메라가 정말 좋고 배터리도 오래가요. 다만 가격이 비싸고 무게가 좀 있네요."

In [ ]:
# ====================================
# 정답 코드
# ====================================

def analyze_review(review_text):
    """
    제품 리뷰를 분석하여 구조화된 JSON으로 반환
    
    Parameters:
    - review_text: 분석할 리뷰 텍스트
    
    Returns:
    - dict: 제품명, 평점, 장점, 단점 포함
    """
    # JSON 형식을 명확히 지정하는 프롬프트
    prompt = f"""다음 제품 리뷰를 분석하여 JSON 형식으로 반환하세요.

리뷰: "{review_text}"

다음 JSON 형식으로만 응답하세요 (다른 텍스트 없이):
{{
    "product": "제품명 (리뷰에서 추출)",
    "rating": 1에서 5 사이의 정수 (리뷰 톤 기반 추정),
    "pros": ["장점1", "장점2"],
    "cons": ["단점1", "단점2"]
}}"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            # 시스템 메시지로 JSON 전용 API 역할 부여
            {"role": "system", "content": "당신은 JSON 형식으로만 응답하는 리뷰 분석 API입니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0  # 일관된 JSON 구조를 위해
    )
    
    # JSON 파싱 시도
    try:
        result = json.loads(response.choices[0].message.content)
        return result
    except json.JSONDecodeError:
        # 파싱 실패 시 원본 반환
        return {
            "error": "JSON 파싱 실패",
            "raw_response": response.choices[0].message.content
        }

# 테스트
test_review = "갤럭시 S24 사용 중인데, 카메라가 정말 좋고 배터리도 오래가요. 다만 가격이 비싸고 무게가 좀 있네요."

result = analyze_review(test_review)

print("=== 리뷰 분석 결과 ===")
print(json.dumps(result, ensure_ascii=False, indent=2))

In [ ]:
# ====================================
# 테스트/검증
# ====================================

# 필수 키 확인
required_keys = ["product", "rating", "pros", "cons"]
for key in required_keys:
    assert key in result, f"필수 키 '{key}'가 없습니다."

# 데이터 타입 확인
assert isinstance(result["product"], str), "product는 문자열이어야 합니다."
assert isinstance(result["rating"], int), "rating은 정수여야 합니다."
assert 1 <= result["rating"] <= 5, "rating은 1~5 사이여야 합니다."
assert isinstance(result["pros"], list), "pros는 리스트여야 합니다."
assert isinstance(result["cons"], list), "cons는 리스트여야 합니다."

# 내용 검증
assert len(result["pros"]) >= 1, "장점이 하나 이상 있어야 합니다."
assert len(result["cons"]) >= 1, "단점이 하나 이상 있어야 합니다."

print("모든 검증 통과!")
print(f"  제품명: {result['product']}")
print(f"  평점: {result['rating']}/5")
print(f"  장점: {result['pros']}")
print(f"  단점: {result['cons']}")

### 풀이 설명

**접근 방법**:
1. JSON 형식을 프롬프트에 명확히 지정
2. 시스템 메시지로 "JSON만 출력" 역할 부여
3. `json.loads()`로 파싱하여 딕셔너리로 변환
4. 파싱 실패 시 에러 처리

**핵심 개념 - JSON 출력 유도**:

```python
# 방법 1: 프롬프트에 형식 명시
prompt = "다음 JSON 형식으로만 응답하세요: {...}"

# 방법 2: 시스템 메시지 활용
messages = [
    {"role": "system", "content": "JSON 형식으로만 응답하세요."},
    {"role": "user", "content": "..."}
]

# 방법 3: OpenAI의 JSON mode (권장)
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    response_format={"type": "json_object"}
)
```

**에러 처리 패턴**:
```python
try:
    result = json.loads(response_text)
except json.JSONDecodeError:
    # 에러 처리
```

**실무 팁**:
- `response_format={"type": "json_object"}` 사용 시 JSON 보장
- 반드시 프롬프트에 "JSON" 언급 필요
- `temperature=0.0`으로 일관된 구조 유지

---

## Q9. 텍스트 분석 봇 (종합) ⭐⭐⭐⭐

**문제**: 긴 텍스트를 입력받아 다음을 수행하는 함수를 작성하세요.
1. 핵심 키워드 3개 추출
2. 한 문장 요약
3. 긍/부정 톤 판단

결과는 딕셔너리로 반환하세요.

In [ ]:
# ====================================
# 정답 코드
# ====================================

def analyze_text(text):
    """
    텍스트를 종합 분석하여 키워드, 요약, 톤을 반환
    
    Parameters:
    - text: 분석할 텍스트
    
    Returns:
    - dict: keywords, summary, tone 포함
    """
    # 모든 분석을 한 번의 API 호출로 처리 (비용 효율)
    prompt = f"""다음 텍스트를 분석하세요.

텍스트:
"{text}"

다음 JSON 형식으로만 응답하세요:
{{
    "keywords": ["키워드1", "키워드2", "키워드3"],
    "summary": "한 문장 요약",
    "tone": "positive 또는 negative 또는 neutral",
    "tone_reason": "톤 판단 이유"
}}"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "텍스트 분석 전문가입니다. JSON 형식으로만 응답합니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0
    )
    
    try:
        result = json.loads(response.choices[0].message.content)
        return result
    except json.JSONDecodeError:
        return {
            "error": "JSON 파싱 실패",
            "raw": response.choices[0].message.content
        }

# 테스트
test_text = """최근 출시된 AI 스피커 신제품이 큰 인기를 끌고 있습니다. 
이 제품은 기존 모델보다 음성 인식 정확도가 30% 향상되었으며, 
스마트홈 연동 기능도 강화되었습니다. 다만 가격이 전작 대비 20% 인상되어 
소비자들의 부담이 커질 것으로 예상됩니다."""

result = analyze_text(test_text)

print("=== 텍스트 분석 결과 ===")
print(json.dumps(result, ensure_ascii=False, indent=2))

In [ ]:
# ====================================
# 테스트/검증
# ====================================

# 필수 키 확인
assert "keywords" in result, "keywords 키가 없습니다."
assert "summary" in result, "summary 키가 없습니다."
assert "tone" in result, "tone 키가 없습니다."

# 키워드 개수 확인
assert len(result["keywords"]) == 3, f"키워드가 3개여야 합니다. 현재: {len(result['keywords'])}개"

# 요약 길이 확인 (한 문장)
assert len(result["summary"]) < 200, "요약이 너무 깁니다."

# 톤 유효성 확인
valid_tones = ["positive", "negative", "neutral"]
assert result["tone"] in valid_tones, f"유효하지 않은 톤: {result['tone']}"

print("모든 검증 통과!")
print(f"\n키워드: {result['keywords']}")
print(f"요약: {result['summary']}")
print(f"톤: {result['tone']}")
if "tone_reason" in result:
    print(f"이유: {result['tone_reason']}")

### 풀이 설명

**접근 방법**:
1. 여러 분석을 하나의 프롬프트로 통합 (API 호출 비용 절감)
2. JSON 형식으로 구조화된 출력 요청
3. 각 분석 항목의 형식 명확히 지정

**핵심 개념 - 복합 분석 패턴**:

```python
# 비효율적: 여러 번 API 호출
keywords = get_keywords(text)   # API 호출 1
summary = get_summary(text)     # API 호출 2
tone = get_tone(text)           # API 호출 3

# 효율적: 한 번의 API 호출로 모두 처리
result = analyze_text(text)     # API 호출 1
```

**JSON 출력 유도 전략**:
1. 시스템 메시지: "JSON 형식으로만 응답"
2. 프롬프트에 정확한 JSON 구조 제시
3. `temperature=0.0`으로 일관성 확보

**실무 팁**:
- 복합 분석 시 비용 대비 효율 고려
- JSON 파싱 실패 대비 에러 처리 필수
- `tone_reason` 같은 추가 필드로 해석 근거 확인

---

## Q10. 다중 리뷰 분류 파이프라인 (종합) ⭐⭐⭐⭐⭐

**문제**: 여러 개의 고객 리뷰를 받아 감성 분석 후 pandas DataFrame으로 결과를 반환하는 파이프라인을 구축하세요.

요구사항:
1. 각 리뷰의 감성(긍정/부정/중립)과 신뢰도(0~1) 분석
2. 결과를 DataFrame으로 정리
3. 감성별 통계 출력

In [ ]:
# ====================================
# 정답 코드
# ====================================

def analyze_single_review(review):
    """
    단일 리뷰 감성 분석
    
    Returns:
    - dict: sentiment, confidence 포함
    """
    prompt = f"""다음 리뷰의 감성을 분석하세요.

리뷰: "{review}"

다음 JSON 형식으로만 응답하세요:
{{
    "sentiment": "positive" 또는 "negative" 또는 "neutral",
    "confidence": 0.0에서 1.0 사이의 신뢰도
}}"""
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "감성 분석 API입니다. JSON만 출력합니다."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.0
    )
    
    try:
        return json.loads(response.choices[0].message.content)
    except json.JSONDecodeError:
        return {"sentiment": "unknown", "confidence": 0.0}


def analyze_reviews(reviews):
    """
    여러 리뷰를 분석하여 DataFrame 반환
    
    Parameters:
    - reviews: 리뷰 텍스트 리스트
    
    Returns:
    - pd.DataFrame: 리뷰, 감성, 신뢰도 포함
    """
    results = []
    
    for i, review in enumerate(reviews, 1):
        print(f"분석 중: {i}/{len(reviews)}...", end="\r")
        
        # 개별 리뷰 분석
        analysis = analyze_single_review(review)
        
        results.append({
            "review": review,
            "sentiment": analysis.get("sentiment", "unknown"),
            "confidence": analysis.get("confidence", 0.0)
        })
    
    print(f"분석 완료: {len(reviews)}개 리뷰")
    
    # DataFrame 생성
    df = pd.DataFrame(results)
    
    # 감성 한글화 (선택)
    sentiment_map = {
        "positive": "긍정",
        "negative": "부정",
        "neutral": "중립",
        "unknown": "분석실패"
    }
    df["sentiment_kr"] = df["sentiment"].map(sentiment_map)
    
    return df

In [ ]:
# 테스트 실행
test_reviews = [
    "정말 좋은 제품이에요! 강력 추천합니다.",
    "배송이 너무 느려요. 일주일이나 걸렸습니다.",
    "가격 대비 괜찮은 품질입니다.",
    "AS 서비스가 최악이에요. 다시는 안 삽니다.",
    "디자인이 예쁘고 사용하기 편해요!"
]

# 분석 실행
result_df = analyze_reviews(test_reviews)

print("\n=== 분석 결과 DataFrame ===")
print(result_df.to_string(index=False))

In [ ]:
# ====================================
# 통계 분석
# ====================================

print("\n=== 감성별 통계 ===")

# 감성 분포
sentiment_counts = result_df["sentiment_kr"].value_counts()
print("\n[감성 분포]")
print(sentiment_counts)

# 감성별 평균 신뢰도
avg_confidence = result_df.groupby("sentiment_kr")["confidence"].mean()
print("\n[감성별 평균 신뢰도]")
print(avg_confidence.round(3))

# 전체 통계 요약
total = len(result_df)
positive_pct = (result_df["sentiment"] == "positive").mean() * 100
negative_pct = (result_df["sentiment"] == "negative").mean() * 100
neutral_pct = (result_df["sentiment"] == "neutral").mean() * 100

print("\n[요약]")
print(f"총 리뷰 수: {total}개")
print(f"긍정 비율: {positive_pct:.1f}%")
print(f"부정 비율: {negative_pct:.1f}%")
print(f"중립 비율: {neutral_pct:.1f}%")
print(f"평균 신뢰도: {result_df['confidence'].mean():.3f}")

In [ ]:
# ====================================
# 테스트/검증
# ====================================

# DataFrame 구조 확인
assert isinstance(result_df, pd.DataFrame), "결과가 DataFrame이 아닙니다."
assert len(result_df) == len(test_reviews), "리뷰 수와 결과 수가 다릅니다."

# 필수 컬럼 확인
required_columns = ["review", "sentiment", "confidence"]
for col in required_columns:
    assert col in result_df.columns, f"필수 컬럼 '{col}'이 없습니다."

# 신뢰도 범위 확인
assert result_df["confidence"].between(0, 1).all(), "신뢰도가 0~1 범위를 벗어났습니다."

# 감성 유효값 확인
valid_sentiments = ["positive", "negative", "neutral", "unknown"]
assert result_df["sentiment"].isin(valid_sentiments).all(), "유효하지 않은 감성 값이 있습니다."

print("모든 검증 통과!")

### 풀이 설명

**접근 방법**:
1. 단일 리뷰 분석 함수 구현 (`analyze_single_review`)
2. 리스트 순회하며 각 리뷰 분석
3. 결과를 DataFrame으로 변환
4. 통계 분석 수행

**핵심 개념 - 파이프라인 패턴**:

```
입력 (리뷰 리스트)
    |
    v
개별 처리 (API 호출)
    |
    v
결과 수집 (리스트)
    |
    v
구조화 (DataFrame)
    |
    v
분석/시각화
```

**최적화 고려사항**:

| 방식 | 장점 | 단점 |
|------|------|------|
| 개별 호출 | 간단, 에러 격리 | 느림, 비용 높음 |
| 배치 처리 | 빠름, 비용 효율 | 구현 복잡, 에러 전파 |
| 비동기 | 매우 빠름 | 구현 가장 복잡 |

**실무 팁**:
- 대량 처리 시 배치(batch) 또는 비동기(async) 방식 고려
- API 호출 실패 시 재시도 로직 추가
- 진행 상황 표시로 사용자 경험 개선
- 결과 캐싱으로 중복 호출 방지

---

---

## 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 내용 | 실무 활용 |
|------|----------|----------|
| 생성형 AI | 패턴 인식을 넘어 패턴 창조 | ChatGPT, DALL-E, Copilot |
| LLM | Transformer 기반 대규모 언어 모델 | GPT(생성), BERT(이해), T5(변환) |
| Self-Attention | 단어 간 관계를 병렬로 계산 | 장거리 의존성 해결 |
| OpenAI API | `client.chat.completions.create()` | 애플리케이션 개발 |
| temperature | 0~2, 창의성 조절 | 0: 결정적, 높을수록 창의적 |

### Part 2: 심화 핵심 요약

| 개념 | 핵심 기법 | 언제 사용? |
|------|----------|----------|
| 스케일링 법칙 | 모델/데이터/컴퓨팅 균형 증가 | LLM 성능 예측 |
| 프롬프트 엔지니어링 | 명확성, 역할 부여, 구조화 | 원하는 출력 유도 |
| Few-shot | 예시 제공 | 일관된 형식, 분류 |
| JSON 출력 | 형식 지정 프롬프트 | 데이터 파이프라인 |

### 실무 팁

1. **API 키 보안**: 환경변수 또는 .env 파일 사용, 절대 코드에 직접 입력 금지
2. **temperature 선택**: 분석/코드는 낮게(0~0.3), 창작은 높게(0.8+)
3. **토큰 비용 관리**: max_tokens 적절히 설정, 불필요한 출력 제한
4. **환각 주의**: 중요한 정보는 반드시 사람이 검증
5. **프롬프트 테스트**: 여러 번 실행하여 일관성 확인